<a href="https://colab.research.google.com/github/SoufianeNoubir/Soufiane-Noubir-EEG-EPQ-2020-2021/blob/main/Spatial_Imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install mne


In [7]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
cd /content/drive/My Drive/EPQ datasets/BCICIV_2a_gdf.zip (Unzipped Files)


/content/drive/My Drive/EPQ datasets/BCICIV_2a_gdf.zip (Unzipped Files)


In [9]:
!pwd

/content/drive/My Drive/EPQ datasets/BCICIV_2a_gdf.zip (Unzipped Files)


In [49]:
import os
import numpy as np
import mne
from scipy.linalg import eigh
import math
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
#reading in file
raw=mne.io.read_raw_gdf('A01T.gdf')
#getting the data
data=raw.get_data()
#getting the labels
events_A, event_id_A = mne.events_from_annotations(raw)

Extracting EDF parameters from /content/drive/My Drive/EPQ datasets/BCICIV_2a_gdf.zip (Unzipped Files)/A01T.gdf...
GDF file detected
Setting channel info structure...
Creating raw.info structure...


/usr/local/lib/python3.7/dist-packages/mne/io/edf/edf.py:1044: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  etmode = np.fromstring(etmode, UINT8).tolist()[0]
<ipython-input-49-0d7db38feb49>:12: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  raw=mne.io.read_raw_gdf('A01T.gdf')


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']


In [50]:
#splitting the dataset up by label
rejections=[]
left=[]
right=[]
up=[]
down=[]

for i in events_A:
  if i[2]==1:
    if i[0]%2==0:
      rejections.append((i[0])/2)
    else:
      rejections.append(((i[0])+1)/2)
  if i[2]==7:
    if i[0]%2==0:
      left.append((i[0])/2)
    else:
      left.append(((i[0])+1)/2)
  if i[2]==8:
    if i[0]%2==0:
      right.append((i[0])/2)
    else:
      right.append(((i[0])+1)/2)
  if i[2]==10:
    if i[0]%2==0:
      up.append((i[0])/2)
    else:
      up.append(((i[0])+1)/2)
  if i[2]==9:
    if i[0]%2==0:
      down.append((i[0])/2)
    else:
      down.append(((i[0])+1)/2)

In [51]:
#removes unwanted electrodes
def remove_electrodes(data,numbers):
  output=[]
  for i in range(len(data)):
    if i in numbers:
      
      output.append(data[i])
  
  return output
data=remove_electrodes(data,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21])

In [52]:
#downsamples data by a given factor
def downsample(data,factor):
  output=[]
  current=[]
  for i in data:
    for j in range(len(i)):
      if j%factor==0:
        current.append(i[j])
    output.append(current)
    current=[]
  return output
data_downsample=downsample(data,2)


In [53]:
#splits data up into segments of EEG data corresponding to one of the 4 classes
up_data=[]
down_data=[]
left_data=[]
right_data=[]
current=[]
for i in up:
  if (int(i)-250) not in rejections:
    for j in data_downsample:
      current.append(j[int(i)+125:int(i)+500])
    up_data.append(current)
    current=[]
for i in down:
  if (int(i)-250) not in rejections:
    for j in data_downsample:
      current.append(j[int(i)+125:int(i)+500])
    down_data.append(current)
    current=[]
for i in right:
  if (int(i)-250) not in rejections:
    for j in data_downsample:
      current.append(j[int(i)+125:int(i)+500])
    right_data.append(current)
    current=[]
for i in left:
  if (int(i)-250) not in rejections:
    for j in data_downsample:
      current.append(j[int(i)+125:int(i)+500])
    left_data.append(current)
    current=[]

In [54]:
#splitting of dataset into training and testing data
left_data_evaluation=[]
right_data_evaluation=[]
down_data_evaluation=[]
up_data_evaluation=[]
import random
for i in up_data:
  x=random.randint(1,4)
  if x==4:
    up_data.remove(i)
    up_data_evaluation.append(i)
for i in right_data:
  x=random.randint(1,4)
  if x==4:
    right_data.remove(i)
    right_data_evaluation.append(i)
for i in down_data:
  x=random.randint(1,4)
  if x==4:
    down_data.remove(i)
    down_data_evaluation.append(i)
for i in left_data:
  x=random.randint(1,4)
  if x==4:
    left_data.remove(i)
    left_data_evaluation.append(i)

In [55]:
space_data=[]
space_data.append(up_data)
space_data.append(right_data)
space_data.append(down_data)
space_data.append(left_data)
space_data_evaluation=[]
space_data_evaluation.append(up_data_evaluation)
space_data_evaluation.append(right_data_evaluation)
space_data_evaluation.append(down_data_evaluation)
space_data_evaluation.append(left_data_evaluation)
x_all_test=[]
y_all_test=[]
x_all_test+=up_data_evaluation
x_all_test+=right_data_evaluation
x_all_test+=down_data_evaluation
x_all_test+=left_data_evaluation
for i in up_data_evaluation:
  y_all_test.append(1)
for i in right_data_evaluation:
  y_all_test.append(2)
for i in down_data_evaluation:
  y_all_test.append(3)
for i in left_data_evaluation:
  y_all_test.append(4)

In [56]:
#implementation of standard CSP approach
def csp(data_1,data_2):
  mean_cov_1=np.cov(np.array(data_1[0]))
  
  mean_cov_2=np.cov(np.array(data_2[0]))
 
  for i in data_1[1:]:
    mean_cov_1+=np.cov(np.array(i))
    
  for i in data_2[1:]:
    mean_cov_2+=np.cov(np.array(i))
    
  mean_cov_1=mean_cov_1/len(data_1)
  mean_cov_2=mean_cov_2/len(data_2)

  
  w_1, v_1 = np.linalg.eig(np.dot(np.linalg.inv(mean_cov_2+mean_cov_1),mean_cov_1))
  sort=np.argsort(w_1)
  
  
  return ([v_1[sort[0]],v_1[sort[1]],v_1[sort[2]],v_1[sort[-3]],v_1[sort[-2]],v_1[sort[-1]]])

In [57]:
#implementation of multiclass CSP
def multiclass_csp(data):
  total_elements=0
  for i in data:
    total_elements+=len(i)
  average=[]
  for i in data:
    total=np.array(i[0])
    
    for j in i[1:]:
      
      total+=np.array(j)
    total=total/len(i)
    
      
    average.append(total)
  total_average=len(data[0])*average[0]
  for i in range(len(data[1:])):
    total_average+=len(data[i])*average[i]
  total_average=total_average/total_elements
  within_class=np.dot(data[0][0]-average[0],(data[0][0]-average[0]).transpose())
  for i in range(len(data)):
    
    for j in data[i]:
      within_class+=np.dot(j-average[i],(j-average[i]).transpose())
  within_class-=np.dot(data[0][0]-average[0],(data[0][0]-average[0]).transpose())
  within_class=within_class/total_elements
  between_class=len(data[0])*(np.dot(average[0]-total_average,(average[0]-total_average).transpose()))
  for i in range(len(average[1:])):
    between_class+=len(data[i])*(np.dot(average[i]-total_average,(average[i]-total_average).transpose()))
  between_class=between_class/len(data)
  w_1, v_1 = np.linalg.eig(np.dot(np.linalg.inv(between_class+within_class),between_class))
  sort=np.argsort(w_1)
  return ([v_1[sort[0]],v_1[sort[1]],v_1[sort[2]],v_1[sort[3]],v_1[sort[4]],v_1[sort[5]]])



In [58]:
# implementation of feature extraction using filters generate from CSP
def feature_extraction(up_data,filters):
  features=[]
  for i in filters:
    features.append(math.log(np.var(np.dot(i.transpose(),up_data))))
    
  return features

In [59]:
csp_filters_left=csp(left_data,right_data+up_data+down_data)
csp_filters_right=csp(right_data,left_data+up_data+down_data)
csp_filters_down=csp(down_data,right_data+up_data+left_data)
csp_filters_up=csp(up_data,right_data+left_data+down_data)
csp_filters_up_down=csp(up_data,down_data)
csp_filters_down_right=csp(right_data,down_data)
csp_filters_up_right=csp(up_data,right_data)
csp_filters_down_left=csp(left_data,down_data)
csp_filters_right_left=csp(right_data,left_data)
csp_filters_up_left=csp(up_data,left_data)
csp_filters_v_h=csp(up_data+down_data,right_data+left_data)
csp_multiclass=multiclass_csp(space_data)

In [60]:
x_vertical_horizontal_train=[]
y_vertical_horizontal_train=[]
x_vertical_horizontal_test=[]
y_vertical_horizontal_test=[]
x_up_down_train=[]
y_up_down_train=[]
x_up_down_test=[]
y_up_down_test=[]
x_left_right_train=[]
y_left_right_train=[]
x_left_right_test=[]
y_left_right_test=[]
x_left_train=[]
y_left_train=[]
x_left_test=[]
y_left_test=[]
x_right_train=[]
y_right_train=[]
x_right_test=[]
y_right_test=[]
x_up_train=[]
y_up_train=[]
x_up_test=[]
y_up_test=[]
x_down_train=[]
y_down_train=[]
x_down_test=[]
y_down_test=[]
x_right_up_train=[]
y_right_up_train=[]
x_right_up_test=[]
y_right_up_test=[]
x_right_down_train=[]
y_right_down_train=[]
x_right_down_test=[]
y_right_down_test=[]
x_left_up_train=[]
y_left_up_train=[]
x_left_up_test=[]
y_left_up_test=[]
x_left_down_train=[]
y_left_down_train=[]
x_left_down_test=[]
y_left_down_test=[]
x_csp_space_train=[]
y_csp_space_train=[]
x_csp_space_test=[]
y_csp_space_test=[]
x_train_nn_space=[]
y_train_nn_space=[]
x_test_nn_space=[]
y_test_nn_space=[]

In [61]:
for i in up_data:
  x_vertical_horizontal_train.append(feature_extraction(i,csp_filters_v_h))
for i in down_data:
  x_vertical_horizontal_train.append(feature_extraction(i,csp_filters_v_h))
for i in right_data:
  x_vertical_horizontal_train.append(feature_extraction(i,csp_filters_v_h))
for i in left_data:
  x_vertical_horizontal_train.append(feature_extraction(i,csp_filters_v_h))

for i in up_data_evaluation:
  x_vertical_horizontal_test.append(feature_extraction(i,csp_filters_v_h))
for i in down_data_evaluation:
  x_vertical_horizontal_test.append(feature_extraction(i,csp_filters_v_h))
for i in right_data_evaluation:
  x_vertical_horizontal_test.append(feature_extraction(i,csp_filters_v_h))
for i in left_data_evaluation:
  x_vertical_horizontal_test.append(feature_extraction(i,csp_filters_v_h))

for i in up_data:
  x_up_down_train.append(feature_extraction(i,csp_filters_up_down))
for i in down_data:
  x_up_down_train.append(feature_extraction(i,csp_filters_up_down))

for i in up_data_evaluation:
  x_up_down_test.append(feature_extraction(i,csp_filters_up_down))
for i in down_data_evaluation:
  x_up_down_test.append(feature_extraction(i,csp_filters_up_down))

for i in right_data:
  x_left_right_train.append(feature_extraction(i,csp_filters_right_left))
for i in left_data:
  x_left_right_train.append(feature_extraction(i,csp_filters_right_left))

for i in right_data_evaluation:
  x_left_right_test.append(feature_extraction(i,csp_filters_right_left))
for i in left_data_evaluation:
  x_left_right_test.append(feature_extraction(i,csp_filters_right_left))

for i in right_data:
  x_right_up_train.append(feature_extraction(i,csp_filters_up_right))
for i in up_data:
  x_right_up_train.append(feature_extraction(i,csp_filters_up_right))

for i in right_data_evaluation:
  x_right_up_test.append(feature_extraction(i,csp_filters_up_right))
for i in up_data_evaluation:
  x_right_up_test.append(feature_extraction(i,csp_filters_up_right))

for i in right_data:
  x_right_down_train.append(feature_extraction(i,csp_filters_down_right))
for i in down_data:
  x_right_down_train.append(feature_extraction(i,csp_filters_down_right))

for i in right_data_evaluation:
  x_right_down_test.append(feature_extraction(i,csp_filters_down_right))
for i in down_data_evaluation:
  x_right_down_test.append(feature_extraction(i,csp_filters_down_right))

for i in left_data:
  x_left_up_train.append(feature_extraction(i,csp_filters_up_left))
for i in up_data:
  x_left_up_train.append(feature_extraction(i,csp_filters_up_left))

for i in left_data_evaluation:
  x_left_up_test.append(feature_extraction(i,csp_filters_up_left))
for i in up_data_evaluation:
  x_left_up_test.append(feature_extraction(i,csp_filters_up_left))


for i in left_data:
  x_left_down_train.append(feature_extraction(i,csp_filters_down_left))
for i in down_data:
  x_left_down_train.append(feature_extraction(i,csp_filters_down_left))

for i in left_data_evaluation:
  x_left_down_test.append(feature_extraction(i,csp_filters_down_left))
for i in down_data_evaluation:
  x_left_down_test.append(feature_extraction(i,csp_filters_down_left))


for i in up_data:
  x_left_train.append(feature_extraction(i,csp_filters_left))
for i in down_data:
  x_left_train.append(feature_extraction(i,csp_filters_left))
for i in right_data:
  x_left_train.append(feature_extraction(i,csp_filters_left))
for i in left_data:
  x_left_train.append(feature_extraction(i,csp_filters_left))

for i in up_data_evaluation:
  x_left_test.append(feature_extraction(i,csp_filters_left))
for i in down_data_evaluation:
  x_left_test.append(feature_extraction(i,csp_filters_left))
for i in right_data_evaluation:
  x_left_test.append(feature_extraction(i,csp_filters_left))
for i in left_data_evaluation:
  x_left_test.append(feature_extraction(i,csp_filters_left))

for i in up_data:
  x_right_train.append(feature_extraction(i,csp_filters_right))
for i in down_data:
  x_right_train.append(feature_extraction(i,csp_filters_right))
for i in right_data:
  x_right_train.append(feature_extraction(i,csp_filters_right))
for i in left_data:
  x_right_train.append(feature_extraction(i,csp_filters_right))

for i in up_data_evaluation:
  x_right_test.append(feature_extraction(i,csp_filters_right))
for i in down_data_evaluation:
  x_right_test.append(feature_extraction(i,csp_filters_right))
for i in right_data_evaluation:
  x_right_test.append(feature_extraction(i,csp_filters_right))
for i in left_data_evaluation:
  x_right_test.append(feature_extraction(i,csp_filters_right))

for i in up_data:
  x_up_train.append(feature_extraction(i,csp_filters_up))
for i in down_data:
  x_up_train.append(feature_extraction(i,csp_filters_up))
for i in right_data:
  x_up_train.append(feature_extraction(i,csp_filters_up))
for i in left_data:
  x_up_train.append(feature_extraction(i,csp_filters_up))

for i in up_data_evaluation:
  x_up_test.append(feature_extraction(i,csp_filters_up))
for i in down_data_evaluation:
  x_up_test.append(feature_extraction(i,csp_filters_up))
for i in right_data_evaluation:
  x_up_test.append(feature_extraction(i,csp_filters_up))
for i in left_data_evaluation:
  x_up_test.append(feature_extraction(i,csp_filters_up))

for i in up_data:
  x_down_train.append(feature_extraction(i,csp_filters_down))
for i in down_data:
  x_down_train.append(feature_extraction(i,csp_filters_down))
for i in right_data:
  x_down_train.append(feature_extraction(i,csp_filters_down))
for i in left_data:
  x_down_train.append(feature_extraction(i,csp_filters_down))

for i in up_data_evaluation:
  x_down_test.append(feature_extraction(i,csp_filters_down))
for i in down_data_evaluation:
  x_down_test.append(feature_extraction(i,csp_filters_down))
for i in right_data_evaluation:
  x_down_test.append(feature_extraction(i,csp_filters_down))
for i in left_data_evaluation:
  x_down_test.append(feature_extraction(i,csp_filters_down))

for i in up_data:
  x_csp_space_train.append(feature_extraction(i,csp_multiclass))
for i in down_data:
  x_csp_space_train.append(feature_extraction(i,csp_multiclass))
for i in right_data:
  x_csp_space_train.append(feature_extraction(i,csp_multiclass))
for i in left_data:
  x_csp_space_train.append(feature_extraction(i,csp_multiclass))

for i in up_data_evaluation:
  x_csp_space_test.append(feature_extraction(i,csp_multiclass))
for i in down_data_evaluation:
  x_csp_space_test.append(feature_extraction(i,csp_multiclass))
for i in right_data_evaluation:
  x_csp_space_test.append(feature_extraction(i,csp_multiclass))
for i in left_data_evaluation:
  x_csp_space_test.append(feature_extraction(i,csp_multiclass))


for i in space_data:
  
  for j in i:
    intermediate=np.transpose(j)
    intermediate_3=[]
    for k in intermediate:
      intermediate_2=[]
      for l in k:
        intermediate_2.append(np.array([l]))
      intermediate_3.append(np.array(intermediate_2))
    x_train_nn_space.append(np.array(intermediate_3))
for i in up_data:
  y_train_nn_space.append(np.array([1,0,0,0]))
for i in right_data:
  y_train_nn_space.append(np.array([0,1,0,0]))
for i in down_data:
  y_train_nn_space.append(np.array([0,0,1,0]))
for i in left_data:
  y_train_nn_space.append(np.array([0,0,0,1]))

for i in space_data_evaluation:
  
  for j in i:
    intermediate=np.transpose(j)
    intermediate_3=[]
    for k in intermediate:
      intermediate_2=[]
      for l in k:
        intermediate_2.append(np.array([l]))
      intermediate_3.append(np.array(intermediate_2))   
    
    x_test_nn_space.append(np.array(intermediate_3))
    
for i in up_data_evaluation:
  y_test_nn_space.append(np.array([1,0,0,0]))
for i in right_data_evaluation:
  y_test_nn_space.append(np.array([0,1,0,0]))
for i in down_data_evaluation:
  y_test_nn_space.append(np.array([0,0,1,0]))
for i in left_data_evaluation:
  y_test_nn_space.append(np.array([0,0,0,1]))


In [62]:
for i in up_data:
 y_vertical_horizontal_train.append(1)
for i in down_data:
  y_vertical_horizontal_train.append(1)
for i in right_data:
  y_vertical_horizontal_train.append(2)
for i in left_data:
  y_vertical_horizontal_train.append(2)

for i in up_data_evaluation:
  y_vertical_horizontal_test.append(1)
for i in down_data_evaluation:
  y_vertical_horizontal_test.append(1)
for i in right_data_evaluation:
  y_vertical_horizontal_test.append(2)
for i in left_data_evaluation:
  y_vertical_horizontal_test.append(2)

for i in up_data:
  y_up_down_train.append(1)
for i in down_data:
  y_up_down_train.append(3)

for i in up_data_evaluation:
  y_up_down_test.append(1)
for i in down_data_evaluation:
  y_up_down_test.append(3)

for i in right_data:
  y_left_right_train.append(2)
for i in left_data:
  y_left_right_train.append(4)

for i in right_data_evaluation:
  y_left_right_test.append(2)
for i in left_data_evaluation:
  y_left_right_test.append(4)

for i in right_data:
  y_right_up_train.append(2)
for i in up_data:
  y_right_up_train.append(1)

for i in right_data_evaluation:
  y_right_up_test.append(2)
for i in up_data_evaluation:
  y_right_up_test.append(1)

for i in right_data:
  y_right_down_train.append(2)
for i in down_data:
  y_right_down_train.append(3)

for i in right_data_evaluation:
  y_right_down_test.append(2)
for i in down_data_evaluation:
  y_right_down_test.append(3)

for i in left_data:
  y_left_up_train.append(4)
for i in up_data:
  y_left_up_train.append(1)

for i in left_data_evaluation:
  y_left_up_test.append(4)
for i in up_data_evaluation:
  y_left_up_test.append(1)

for i in left_data:
  y_left_down_train.append(4)
for i in down_data:
  y_left_down_train.append(3)

for i in left_data_evaluation:
  y_left_down_test.append(4)
for i in down_data_evaluation:
  y_left_down_test.append(3)

for i in up_data:
  y_left_train.append(2)
for i in down_data:
  y_left_train.append(2)
for i in right_data:
  y_left_train.append(2)
for i in left_data:
  y_left_train.append(1)

for i in up_data_evaluation:
  y_left_test.append(2)
for i in down_data_evaluation:
  y_left_test.append(2)
for i in right_data_evaluation:
  y_left_test.append(2)
for i in left_data_evaluation:
  y_left_test.append(1)

for i in up_data:
  y_right_train.append(2)
for i in down_data:
  y_right_train.append(2)
for i in right_data:
  y_right_train.append(1)
for i in left_data:
  y_right_train.append(2)

for i in up_data_evaluation:
  y_right_test.append(2)
for i in down_data_evaluation:
  y_right_test.append(2)
for i in right_data_evaluation:
  y_right_test.append(1)
for i in left_data_evaluation:
  y_right_test.append(2)

for i in up_data:
  y_up_train.append(1)
for i in down_data:
  y_up_train.append(2)
for i in right_data:
  y_up_train.append(2)
for i in left_data:
  y_up_train.append(2)

for i in up_data_evaluation:
  y_up_test.append(1)
for i in down_data_evaluation:
  y_up_test.append(2)
for i in right_data_evaluation:
  y_up_test.append(2)
for i in left_data_evaluation:
  y_up_test.append(2)

for i in up_data:
  y_down_train.append(2)
for i in down_data:
  y_down_train.append(1)
for i in right_data:
  y_down_train.append(2)
for i in left_data:
  y_down_train.append(2)

for i in up_data_evaluation:
  y_down_test.append(2)
for i in down_data_evaluation:
  y_down_test.append(1)
for i in right_data_evaluation:
  y_down_test.append(2)
for i in left_data_evaluation:
  y_down_test.append(2)

for i in up_data:
  y_csp_space_train.append(1)
for i in down_data:
  y_csp_space_train.append(3)
for i in right_data:
  y_csp_space_train.append(2)
for i in left_data:
  y_csp_space_train.append(4)

for i in up_data_evaluation:
  y_csp_space_test.append(1)
for i in down_data_evaluation:
  y_csp_space_test.append(3)
for i in right_data_evaluation:
  y_csp_space_test.append(2)
for i in left_data_evaluation:
  y_csp_space_test.append(4)

In [63]:
#uses one_v_one classification to output the predicted class
def one_v_one(outputs):
  num_1=outputs.count(1)
  num_2=outputs.count(2)
  num_3=outputs.count(3)
  num_4=outputs.count(4)
  num=[num_1,num_2,num_3,num_4]
  if num.count(max(num))==1:
    return (num.index(max(num)))+1
  else:
    
    winners=[]
    for i in range(len(num)):
      if num[i] ==max(num):
        winners.append(i+1)
    if len(winners)==3:
      return ("alert")
    else:
      if min(winners)==1:
        return outputs[sum(winners)-3]
      else:
        return outputs[sum(winners)-2]
#uses one_v_many classification to output the predicted class
def one_v_many(probs):
  return (probs.index(max(probs))+1)
#uses binary search to output the predicted class
def binary_search(outputs):
  if outputs[0]==1:
    return outputs[1]
  else:
    return outputs[2]
#returns accuracy
def accuracy(outputs,correct):
  right=0
  for i in range(len(outputs)):
    if outputs[i]==correct[i]:
      right+=1
  return right/(len(outputs))

In [ ]:
multiclass_LDA = LinearDiscriminantAnalysis()
multiclass_LDA.fit(x_csp_space_train,y_csp_space_train)
print(multiclass_LDA.score(x_csp_space_test,y_csp_space_test))

LDA_binary_search_v_h = LinearDiscriminantAnalysis()
LDA_binary_search_v_h.fit(x_vertical_horizontal_train,y_vertical_horizontal_train)
print(LDA_binary_search_v_h.score(x_vertical_horizontal_test,y_vertical_horizontal_test))

LDA_ovo_u_d = LinearDiscriminantAnalysis()
LDA_ovo_u_d.fit(x_up_down_train,y_up_down_train)
print(LDA_ovo_u_d.score(x_up_down_test,y_up_down_test))

LDA_ovo_u_r = LinearDiscriminantAnalysis()
LDA_ovo_u_r .fit(x_right_up_train,y_right_up_train)
print(LDA_ovo_u_r .score(x_right_up_test,y_right_up_test))

LDA_ovo_u_l = LinearDiscriminantAnalysis()
LDA_ovo_u_l.fit(x_left_up_train,y_left_up_train)
print(LDA_ovo_u_l.score(x_left_up_test,y_left_up_test))

LDA_ovo_d_r = LinearDiscriminantAnalysis()
LDA_ovo_d_r.fit(x_right_down_train,y_right_down_train)
print(LDA_ovo_d_r.score(x_right_down_test,y_right_down_test))

LDA_ovo_d_l = LinearDiscriminantAnalysis()
LDA_ovo_d_l.fit(x_left_down_train,y_left_down_train)
print(LDA_ovo_d_l.score(x_left_down_test,y_left_down_test))

LDA_ovo_r_l = LinearDiscriminantAnalysis()
LDA_ovo_r_l.fit(x_left_right_train,y_left_right_train)
print(LDA_ovo_r_l.score(x_left_right_test,y_left_right_test))

LDA_ovm_u = LinearDiscriminantAnalysis()
LDA_ovm_u.fit(x_up_train,y_up_train)
print(LDA_ovm_u.score(x_up_test,y_up_test))

LDA_ovm_r = LinearDiscriminantAnalysis()
LDA_ovm_r.fit(x_right_train,y_right_trai76n)
print(LDA_ovm_r.score(x_right_test,y_right_test))

LDA_ovm_d = LinearDiscriminantAnalysis()
LDA_ovm_d.fit(x_down_train,y_down_train)
print(LDA_ovm_d.score(x_down_test,y_down_test))

LDA_ovm_l = LinearDiscriminantAnalysis()
LDA_ovm_l.fit(x_left_train,y_left_train)
print(LDA_ovm_l.score(x_left_test,y_left_test))



In [ ]:
multiclass_SVM = svm.SVC(probability=True)
multiclass_SVM.fit(x_csp_space_train,y_csp_space_train)
print(multiclass_SVM.score(x_csp_space_test,y_csp_space_test))

SVM_binary_search_v_h = svm.SVC(probability=True)
SVM_binary_search_v_h.fit(x_vertical_horizontal_train,y_vertical_horizontal_train)
print(SVM_binary_search_v_h.score(x_vertical_horizontal_test,y_vertical_horizontal_test))

SVM_ovo_u_d = svm.SVC(probability=True)
SVM_ovo_u_d.fit(x_up_down_train,y_up_down_train)
print(SVM_ovo_u_d.score(x_up_down_test,y_up_down_test))

SVM_ovo_u_r = svm.SVC(probability=True)
SVM_ovo_u_r .fit(x_right_up_train,y_right_up_train)
print(SVM_ovo_u_r .score(x_right_up_test,y_right_up_test))

SVM_ovo_u_l = svm.SVC(probability=True)
SVM_ovo_u_l.fit(x_left_up_train,y_left_up_train)
print(SVM_ovo_u_l.score(x_left_up_test,y_left_up_test))

SVM_ovo_d_r = svm.SVC(probability=True)
SVM_ovo_d_r.fit(x_right_down_train,y_right_down_train)
print(SVM_ovo_d_r.score(x_right_down_test,y_right_down_test))

SVM_ovo_d_l = svm.SVC(probability=True)
SVM_ovo_d_l.fit(x_left_down_train,y_left_down_train)
print(SVM_ovo_d_l.score(x_left_down_test,y_left_down_test))

SVM_ovo_r_l = svm.SVC(probability=True)
SVM_ovo_r_l.fit(x_left_right_train,y_left_right_train)
print(SVM_ovo_r_l.score(x_left_right_test,y_left_right_test))

SVM_ovm_u = svm.SVC(probability=True)
SVM_ovm_u.fit(x_up_train,y_up_train)
print(SVM_ovm_u.score(x_up_test,y_up_test))

SVM_ovm_r = svm.SVC(probability=True)
SVM_ovm_r.fit(x_right_train,y_right_train)
print(SVM_ovm_r.score(x_right_test,y_right_test))

SVM_ovm_d = svm.SVC(probability=True)
SVM_ovm_d.fit(x_down_train,y_down_train)
print(SVM_ovm_d.score(x_down_test,y_down_test))

SVM_ovm_l = svm.SVC(probability=True)
SVM_ovm_l.fit(x_left_train,y_left_train)
print(SVM_ovm_l.score(x_left_test,y_left_test))

In [ ]:
multiclass_LR = LogisticRegression(multi_class='multinomial', solver='lbfgs')
multiclass_LR.fit(x_csp_space_train,y_csp_space_train)
print(multiclass_LR.score(x_csp_space_test,y_csp_space_test))

LR_binary_search_v_h = LogisticRegression()
LR_binary_search_v_h.fit(x_vertical_horizontal_train,y_vertical_horizontal_train)
print(LR_binary_search_v_h.score(x_vertical_horizontal_test,y_vertical_horizontal_test))

LR_ovo_u_d = LogisticRegression()
LR_ovo_u_d.fit(x_up_down_train,y_up_down_train)
print(LR_ovo_u_d.score(x_up_down_test,y_up_down_test))

LR_ovo_u_r = LogisticRegression()
LR_ovo_u_r .fit(x_right_up_train,y_right_up_train)
print(LR_ovo_u_r .score(x_right_up_test,y_right_up_test))

LR_ovo_u_l = LogisticRegression()
LR_ovo_u_l.fit(x_left_up_train,y_left_up_train)
print(LR_ovo_u_l.score(x_left_up_test,y_left_up_test))

LR_ovo_d_r = LogisticRegression()
LR_ovo_d_r.fit(x_right_down_train,y_right_down_train)
print(LR_ovo_d_r.score(x_right_down_test,y_right_down_test))

LR_ovo_d_l = LogisticRegression()
LR_ovo_d_l.fit(x_left_down_train,y_left_down_train)
print(LR_ovo_d_l.score(x_left_down_test,y_left_down_test))

LR_ovo_r_l = LogisticRegression()
LR_ovo_r_l.fit(x_left_right_train,y_left_right_train)
print(LR_ovo_r_l.score(x_left_right_test,y_left_right_test))

LR_ovm_u = LogisticRegression()
LR_ovm_u.fit(x_up_train,y_up_train)
print(LR_ovm_u.score(x_up_test,y_up_test))

LR_ovm_r = LogisticRegression()
LR_ovm_r.fit(x_right_train,y_right_train)
print(LR_ovm_r.score(x_right_test,y_right_test))

LR_ovm_d = LogisticRegression()
LR_ovm_d.fit(x_down_train,y_down_train)
print(LR_ovm_d.score(x_down_test,y_down_test))

LR_ovm_l = LogisticRegression()
LR_ovm_l.fit(x_left_train,y_left_train)
print(LR_ovm_l.score(x_left_test,y_left_test))

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
#initialising neural network
model = models.Sequential()
#specifying topology of neural network
model.add(layers.Conv2D(22, (11,1), (1,1),input_shape=(375,22,1),padding="valid"))
model.add(layers.LeakyReLU(0.01))
model.add(layers.SpatialDropout2D(.5))
model.add(layers.Conv2D(44, (1,9), (1,1),padding="valid"))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(0.01))
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
model.add(layers.Conv2D(88, (11,1), (1,1),padding="valid"))
model.add(layers.LeakyReLU(0.01))
model.add(layers.SpatialDropout2D(.5))
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
model.add(layers.Conv2D(88, (11,1), (1,1),padding="valid"))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(0.01))
model.add(layers.SpatialDropout2D(.5))
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
model.add(layers.Conv2D(176, (2,1), (1,1),padding="valid"))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(0.01))
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
model.add(layers.Conv2D(352, (2,1), (1,1),padding="valid"))
model.add(layers.BatchNormalization())
model.add(layers.LeakyReLU(0.01))
model.add(layers.MaxPooling2D((2, 1),(2,1),padding="valid"))
model.add(layers.Flatten())
model.add(layers.Dense(4))
model.add(layers.Softmax())
#specifying training mechanism
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()
#training neural network
model.fit(np.array(x_train_nn_space),np.array(y_train_nn_space),epochs=500,validation_data=(np.array(x_test_nn_space), np.array(y_test_nn_space)))

In [67]:
LDA_binary_outputs=[]
LDA_ovo_outputs=[]
LDA_ovm_outputs=[]
LDA_multiclass_outputs=[]
SVM_binary_outputs=[]
SVM_ovo_outputs=[]
SVM_ovm_outputs=[]
SVM_multiclass_outputs=[]
LR_binary_outputs=[]
LR_ovo_outputs=[]
LR_ovm_outputs=[]
LR_multiclass_outputs=[]

In [ ]:
for i in x_all_test:
  current=feature_extraction(i,csp_multiclass)
  LDA_multiclass_outputs.append(multiclass_LDA.predict([current]))
  SVM_multiclass_outputs.append(multiclass_SVM.predict([current]))
  LR_multiclass_outputs.append(multiclass_LR.predict([current]))
print(accuracy(LDA_multiclass_outputs,y_all_test))
print(accuracy(SVM_multiclass_outputs,y_all_test))
print(accuracy(LR_multiclass_outputs,y_all_test))

In [ ]:
for i in x_all_test:
  current_1=feature_extraction(i,csp_filters_up_right)
  current_2=feature_extraction(i,csp_filters_up_down)
  current_3=feature_extraction(i,csp_filters_up_left)
  current_4=feature_extraction(i,csp_filters_down_right)
  current_5=feature_extraction(i,csp_filters_right_left)
  current_6=feature_extraction(i,csp_filters_down_left)
  output_1=LDA_ovo_u_r.predict([current_1])
  output_2=LDA_ovo_u_d.predict([current_2])
  output_3=LDA_ovo_u_l.predict([current_3])
  output_4=LDA_ovo_d_r.predict([current_4])
  output_5=LDA_ovo_r_l.predict([current_5])
  output_6=LDA_ovo_d_l.predict([current_6])
  LDA_ovo_outputs.append(one_v_one([output_1,output_2,output_3,output_4,output_5,output_6]))
  output_1=SVM_ovo_u_r.predict([current_1])
  output_2=SVM_ovo_u_d.predict([current_2])
  output_3=SVM_ovo_u_l.predict([current_3])
  output_4=SVM_ovo_d_r.predict([current_4])
  output_5=SVM_ovo_r_l.predict([current_5])
  output_6=SVM_ovo_d_l.predict([current_6])
  SVM_ovo_outputs.append(one_v_one([output_1,output_2,output_3,output_4,output_5,output_6]))
  output_1=LR_ovo_u_r.predict([current_1])
  output_2=LR_ovo_u_d.predict([current_2])
  output_3=LR_ovo_u_l.predict([current_3])
  output_4=LR_ovo_d_r.predict([current_4])
  output_5=LR_ovo_r_l.predict([current_5])
  output_6=LR_ovo_d_l.predict([current_6])
  LR_ovo_outputs.append(one_v_one([output_1,output_2,output_3,output_4,output_5,output_6]))

print(accuracy(LDA_ovo_outputs,y_all_test))
print(accuracy(SVM_ovo_outputs,y_all_test))
print(accuracy(LR_ovo_outputs,y_all_test))

In [ ]:
for i in x_all_test:
  current_1=feature_extraction(i,csp_filters_v_h)
  current_2=feature_extraction(i,csp_filters_up_down)
  current_3=feature_extraction(i,csp_filters_right_left)
  output_1=LDA_binary_search_v_h.predict([current_1])
  output_2=LDA_ovo_u_d.predict([current_2])
  output_3=LDA_ovo_r_l.predict([current_3])
  LDA_binary_outputs.append(binary_search([output_1,output_2,output_3]))
  output_1=SVM_binary_search_v_h.predict([current_1])
  output_2=SVM_ovo_u_d.predict([current_2])
  output_3=SVM_ovo_r_l.predict([current_3])
  SVM_binary_outputs.append(binary_search([output_1,output_2,output_3]))
  output_1=LR_binary_search_v_h.predict([current_1])
  output_2=LR_ovo_u_d.predict([current_2])
  output_3=LR_ovo_r_l.predict([current_3])
  LR_binary_outputs.append(binary_search([output_1,output_2,output_3]))
print(accuracy(LDA_binary_outputs,y_all_test))
print(accuracy(SVM_binary_outputs,y_all_test))
print(accuracy(LR_binary_outputs,y_all_test))50.0%


In [ ]:
for i in x_all_test:
  current_1=feature_extraction(i,csp_filters_up)
  current_2=feature_extraction(i,csp_filters_down)
  current_3=feature_extraction(i,csp_filters_left)
  current_4=feature_extraction(i,csp_filters_right)
  output_1=(LDA_ovm_u.predict_proba([current]))[0][0]
  output_2=(LDA_ovm_d.predict_proba([current]))[0][0]
  output_3=(LDA_ovm_l.predict_proba([current]))[0][0]
  output_4=(LDA_ovm_r.predict_proba([current]))[0][0]
  LDA_ovm_outputs.append(one_v_many([output_1,output_4,output_2,output_3]))
  output_1=(SVM_ovm_u.predict_proba([current]))[0][0]
  output_2=(SVM_ovm_d.predict_proba([current]))[0][0]
  output_3=(SVM_ovm_l.predict_proba([current]))[0][0]
  output_4=(SVM_ovm_r.predict_proba([current]))[0][0]
  SVM_ovm_outputs.append(one_v_many([output_1,output_4,output_2,output_3]))
  output_1=(LR_ovm_u.predict_proba([current]))[0][0]
  output_2=(LR_ovm_d.predict_proba([current]))[0][0]
  output_3=(LR_ovm_l.predict_proba([current]))[0][0]
  output_4=(LR_ovm_r.predict_proba([current]))[0][0]
  LR_ovm_outputs.append(one_v_many([output_1,output_4,output_2,output_3]))
print(accuracy(LDA_ovm_outputs,y_all_test))
print(accuracy(SVM_ovm_outputs,y_all_test))
print(accuracy(LR_ovm_outputs,y_all_test))


